In [2]:
import json
import os
from qa_processing import verify_qa_list

In [ ]:
qa_list = []
dataset_with_agent_answer = ""
dataset_with_agent_answer_verified = dataset_with_agent_answer.replace(
    "_with_agent_answer", "_with_agent_answer_verified"
)

with open(dataset_with_agent_answer, "r") as f:
    for line in f:
        qa = json.loads(line)
        if os.path.exists(qa["mem_path"]):
            qa_list.append(qa)

qa_list = verify_qa_list(qa_list)

with open(dataset_with_agent_answer_verified, "w") as f:
    for qa in qa_list:
        f.write(json.dumps(qa) + "\n")

In [3]:
# calculate accuracy
dataset_with_agent_answer_verified = "data/annotations/results/small_test_with_agent_answer_verified_0.jsonl"
total = 0
correct = 0

with open(dataset_with_agent_answer_verified, "r") as f:
    for line in f:
        qa = json.loads(line)
        total += 1
        if qa["verify_result"].lower().startswith("yes"):
            correct += 1

print(f"Accuracy: {correct / total}")

Accuracy: 0.5496031746031746


In [10]:
# calculate pass@k
k = 4

dataset_with_agent_answer_verified = (
    "data/annotations/results/small_test_with_agent_answer_verified_{round}.jsonl"
)

results = [dataset_with_agent_answer_verified.format(round=i) for i in range(k)]
result_files = [open(result, "r") for result in results]

total = 0
correct = 0

# Read all QA pairs line by line across k files
while True:
    qa_attempts = []
    # Try to read next line from each file
    for f in result_files:
        line = f.readline()
        if line:
            qa = json.loads(line)
            qa_attempts.append(qa["verify_result"].lower().startswith("yes"))

    # If we've reached end of files, break
    if not qa_attempts:
        break

    # Count as correct if any attempt was successful
    total += 1
    if any(qa_attempts):
        correct += 1

# Close all files
for f in result_files:
    f.close()

print(f"Pass@{k}: {correct / total}")

Pass@4: 0.6944444444444444
